### Defining the board name.

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'NONE'

### Configuration the CWNANO board and its ADC.

In [2]:
%run "../../../Helper_Scripts/Setup_Generic.ipynb"
scope.adc.samples = 50000
scope._cwusb.usbtx.rep = 0x81
scope._cwusb.usbtx.wep = 0x02

OSError: Unable to communicate with found ChipWhisperer. Check that another process isn't connected to it and that you have permission to communicate with it.

OSError: Unable to communicate with found ChipWhisperer. Check that another process isn't connected to it and that you have permission to communicate with it.

### Programming board with the provided HEX file.

In [ ]:
fw_path = "../casino-CWNANO.hex"
cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)

### In this section we call the casino function and also record the power when the repositioning of the array  is happenning. Now we have power traces.
### Also, we save the extracted power reaces in two format (.txt, .npy) as it is required by organizer.


In [ ]:
import numpy as np
import time
from scipy.stats import pearsonr

power=np.zeros(50000, dtype=float)
msg = bytearray([])
scope.arm()
target.simpleserial_write('s',msg)
scope.capture()
power=scope.get_last_trace()[0:50000]

np.save('casino_power_file', power)
file = open("casino_power_file.txt","w")
for i in range(50000):
    file.write(str(power[i]))
    file.write("\n")        
file.close()        

### Here you can load my power traces that I already stored. Please do not run this code if you want to extract power traces. My_casino_power_file.npy and My_casino_power_file.txt is my power traces when I ran code.

In [ ]:
import numpy as np
import time
from scipy.stats import pearsonr

power=np.zeros(50000, dtype=float)
power = np.load('My_casino_power_file.npy')

#f=open("My_casino_power_file.txt","r")
#val = f.readlines()
#for i in range(50000):
#    power[i] = val[i]

### Based on the .c code, in the draw function, in each iteration of first loop, the second loop does multiplication based on the value of the element in array. If the value is bigger, the multiplier iteration gets bigger. Whenever multiply is finished we have considerable drop in power traces. Then we understand that it goes for the next iteration.  

![casino.png](casino.png)

### For example in aboce picture, you can see the location of the drop in power traces. When we detect that point, we start to count the number of spike till the next drop. Here it is clear that after number 20, number 30 are located in the array.

In [ ]:
list_array = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]

signal=np.zeros(50000, dtype=float)
index_c = np.where(power > 0.75*max(power[1:22000]))
signal[index_c[0]] = power[index_c[0]]

array=np.zeros(15, dtype=int)
index = 0;
cnt = 0;
for i in range(40000):
    if(signal[i] != 0):
        cnt = cnt + 1
    if(signal[i+19] != 0 and signal[i] != 0 and signal[i+19+18] != 0):
        array[index] = (cnt - 1)
        index = index + 1
        cnt = 0;
    if(index == 14):
        break
array[14] = list(set(list_array) - set(array))[0]


def tune(data, list_array):
    for i in range(len(list_array)):
        index = 0
        m = max(list_array) + 1
        for j in range(len(list_array)):
            if(abs(data[i] - list_array[j]) < m):
                m = abs(data[i] - list_array[j])
                index = j
        data[i] = list_array[index]
    return data
array = tune(array, list_array)            

### After finding the corrected order of array, we need to check it with verify function. Here we call verify function.

In [ ]:
msg = bytearray(list(array))
target.simpleserial_write('a',bytearray(msg))
done=target.simpleserial_read('r', 1)
if(done == 0):
    array[0] = array[0] - 1
    msg = bytearray(list(array))
    target.simpleserial_write('a',bytearray(msg))
    done=target.simpleserial_read('r', 1)
print("The Corrected Order of Array: {}".format(list(msg)))
print("Response of Verify Function: {}".format(list(done)))

### This is the Answer:
### The Corrected Order of Array : 
### [120, 90, 80, 60, 110, 10, 50, 20, 30, 40, 150, 140, 70, 100, 130]